In [ ]:
#pip install efficientnet

In [10]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, tensorflow.keras.backend as K
import seaborn as sns
import random
import efficientnet.tfkeras as efn
import cv2
from tensorflow.keras import optimizers
import hashlib
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [11]:
def seed_everything(seed=13):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_KERAS'] = '1'
    random.seed(seed)

seed_everything(42)

In [ ]:
def TPU():
    try:
        tpu = tf.distribute.cluster_resolver.TPUCluseterResolver()
        print("Running on tpu", tpu.master())
    except ValueError:
        tpu = None
    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.discrete.experimental.TPUStrategy(tpu)
    else:
        strategy = tf.discrete.get_strategy(tpu)
    print("REPLICAS: ", strategy.num_replicas_in_sync)
    return strategy